In [ ]:
pip install vit-pytorch linformer

     |████████████████████████████████| 52 kB 690 kB/s 


In [ ]:
pip install wandb

     |████████████████████████████████| 1.7 MB 5.3 MB/s 
     |████████████████████████████████| 181 kB 44.4 MB/s 
     |████████████████████████████████| 144 kB 48.7 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8806 sha256=a99bc3b2a581c924e9b395de9864c189120aead79c95eada70aa2d15781ec39e
  Stored in directory: /root/.cache/pip/wheels/3e/31/09/fa59cef12cdcfecc627b3d24273699f390e71828921b2cbba2
Successfully built pathtools


In [ ]:
from __future__ import print_function

import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
# import Linformer

from linformer import Linformer
import glob
from PIL import Image
from itertools import chain
from vit_pytorch.efficient import ViT
from tqdm.notebook import tqdm
# import torch and related libraries

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset, SubsetRandomSampler
import wandb

from torchvision.datasets import CIFAR10



torch.backends.cudnn.benchmarks = True

torch.backends.cudnn.deterministic = True

wandb.login(key='46f2d6a5ffcc458fed2cca6cf446900f97e396e1')

########################################################################################
################################ HYPERPARAMETERS #######################################
########################################################################################

config=dict(
IMAGE_SIZE = 224,
BATCH_SIZE = 128,
EPOCHS = 100,
VAL_SPLIT = 0.2,
LEARNING_RATE = 3e-5,
GAMMA = 0.7,
STEP_SIZE = 1,
saved_path = './models/'
)



wandb.init(project='Vision-Transformer-CIFAR10',config=config)
config = wandb.config



data_transform = transforms.Compose([transforms.Resize((config.IMAGE_SIZE,config.IMAGE_SIZE)),transforms.RandomResizedCrop(config.IMAGE_SIZE),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor()])
dataset = CIFAR10(root="./data",download=True,transform=data_transform)


def train_log(epoch,train_loss,train_acc,test_acc,test_loss):
    wandb.log({"epoch": epoch, "train_loss": train_loss, "train_acc":train_acc,"test_loss":test_loss,"test_acc":test_acc})


def Split_index(n,val_percent):
    n_val = int(val_percent*n)
    index = np.random.permutation(n)
    return index[n_val:],index[:n_val]



train_indices, val_indices = Split_index(len(dataset),val_percent=config.VAL_SPLIT)
print(len(train_indices),len(val_indices))
print(train_indices,val_indices)


train_sampler = SubsetRandomSampler(train_indices)
val_sampler = SubsetRandomSampler(val_indices)

train_dl = DataLoader(dataset,batch_size =config.BATCH_SIZE,sampler=train_sampler,num_workers=4,pin_memory=True)
val_dl = DataLoader(dataset,batch_size =config.BATCH_SIZE,sampler=val_sampler,num_workers=4,pin_memory=True)


efficient_transformer = Linformer(
    dim=128,
    seq_len=49+1,  # 7x7 patches + 1 cls-token
    depth=12,
    heads=8,
    k=64
)

model = ViT(
    dim=128,
    image_size=config.IMAGE_SIZE,
    patch_size=32,
    num_classes=10,
    transformer=efficient_transformer,
    channels=3,
).cuda()


criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=config.LEARNING_RATE)
scheduler = StepLR(optimizer, step_size=config.STEP_SIZE, gamma=config.GAMMA)


History = []
for epoch in range(config.EPOCHS):
    epoch_loss = 0
    epoch_accuracy = 0
    for data, label in tqdm(train_dl):
        data = data.cuda()
        label = label.cuda()
        output = model(data)
        #print(label.size(),output.size())
        #print(output)
        loss = criterion(output, label)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(train_dl)
        epoch_loss += loss / len(train_dl)
    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in val_dl:
            data = data.cuda()
            label = label.cuda()
            val_output = model(data)
            val_loss = criterion(val_output, label)
            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(val_dl)
            epoch_val_loss += val_loss / len(val_dl)
    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )
    train_log(epoch,epoch_loss,epoch_accuracy,epoch_val_accuracy,epoch_val_loss)
    history = {
        'train_loss':epoch_loss,
        'train_acc':epoch_accuracy,
        'val_loss':epoch_val_loss,
        'val_acc':epoch_val_accuracy
    }
    History.append(history)


train_acc = []
train_loss = []
val_acc = []
val_loss = []
for i in range(config.EPOCHS):
  temp1 = History[i]['train_acc'].to('cpu').detach().numpy()
  temp2 = History[i]['train_loss'].to('cpu').detach().numpy()
  temp3 = History[i]['val_acc'].to('cpu').detach().numpy()
  temp4 = History[i]['val_loss'].to('cpu').detach().numpy()

  train_acc.append(temp1)
  train_loss.append(temp2)
  val_acc.append(temp3)
  val_loss.append(temp4)


plt.plot(range(config.EPOCHS),train_acc,label='Train Acc')
plt.plot(range(config.EPOCHS),val_acc,label='Val Acc')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.title("Training and Validation Accuracy")
plt.savefig("Training.png")
plt.show()


plt.plot(range(config.EPOCHS),train_loss,label='Train Loss')
plt.plot(range(config.EPOCHS),val_loss,label='Val Loss')
plt.legend()
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title("Training and Validation Loss")
plt.savefig("Loss.png")
plt.show()


wandb: W&B API key is configured (use `wandb login --relogin` to force relogin)
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: shashi7679 (use `wandb login --relogin` to force relogin)


  0%|          | 0/170498071 [00:00<?, ?it/s]

Extracting ./data/cifar-10-python.tar.gz to ./data
40000 10000
[12967 37593 34521 ...  2799 19744 45141] [34934 30846 28387 ... 46323 18849 43781]


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:481: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


  0%|          | 0/313 [00:00<?, ?it/s]

...

  0%|          | 0/313 [00:00<?, ?it/s]

Epoch : 37 - loss : 1.4277 - acc: 0.4935 - val_loss : 1.4480 - val_acc: 0.4839



  0%|          | 0/313 [00:00<?, ?it/s]

Epoch : 38 - loss : 1.4214 - acc: 0.4924 - val_loss : 1.4611 - val_acc: 0.4778



  0%|          | 0/313 [00:00<?, ?it/s]